In [1]:
import pandas as pd
import plotly.express as px
import sqlite3

DB_PATH = r"..\data\metro_datetime.db"
TABLE = "Metro_Line_1_Schedule"

##### 1. 테이블 속성의 타입을 알아보자.

In [2]:
# Context Manager 사용 (권장)
with sqlite3.connect(DB_PATH) as conn:
    query = f"""
            SELECT *
            FROM {TABLE}
            """
    result = pd.read_sql(query, conn)
display(result.dtypes.to_frame(name='dtype'))

,dtype
고유번호,int64
호선,int64
역사코드,int64
역사명,object
주중주말,object
방향,object
급행여부,int64
열차코드,object
열차도착시간,object
열차출발시간,object


##### 2. 전체 데이터를 확인해보자.

In [3]:
# Context Manager 사용 (권장)
with sqlite3.connect(DB_PATH) as conn:
    query = f"""
            SELECT *
            FROM {TABLE}
            """
    result = pd.read_sql(query, conn)
result.sort_values("열차도착시간_dt")
result.to_csv("text.csv")


##### 3. 전체 역의 수를 알아보자.

In [4]:
# Context Manager 사용 (권장)
with sqlite3.connect(DB_PATH) as conn:
    query = f"""
            SELECT
                COUNT(DISTINCT `역사코드`) as 역의수
            FROM {TABLE}
            """
    result = pd.read_sql(query, conn)
result

,역의수
0,102


##### 6. 운행구간을 알아보자.

In [5]:
# Context Manager 사용 (권장)
with sqlite3.connect(DB_PATH) as conn:
    query = f"""
            SELECT
                `출발역` AS 기점,
                `도착역` AS 종점
            FROM {TABLE}
            GROUP BY `출발역`, `도착역`
            """
    result = pd.read_sql(query, conn)
result

,기점,종점
0,광명,영등포
1,광운대,구로
2,광운대,동두천
3,광운대,병점
4,광운대,서동탄
...,...,...
97,청량리,병점
98,청량리,서동탄
99,청량리,서울역
100,청량리,신창


##### 7. 운행구간별 실제 운행 열차 수

In [6]:
# Context Manager 사용 (권장)
with sqlite3.connect(DB_PATH) as conn:
    query = f"""
            SELECT
                `주중주말`,
                `출발역`,
                `도착역`,
                COUNT(DISTINCT `열차코드`) AS 운행열차수
            FROM {TABLE}
            GROUP BY `주중주말`, `출발역`, `도착역`
            ORDER BY `주중주말`, 운행열차수 DESC
            """
    result = pd.read_sql(query, conn)

week = {
        "DAY": result[result['주중주말'] == 'DAY'],
        "SAT": result[result['주중주말'] == 'SAT'],
        "END": result[result['주중주말'] == 'END']
       }

for key,val in week.items():
    print(f"\n[{key}] 총 {len(val)}개 운행 구간")
    print("-" * 70)
    display(val[['출발역', '도착역', '운행열차수']])


[DAY] 총 97개 운행 구간
----------------------------------------------------------------------


,출발역,도착역,운행열차수
0,동인천,용산,78
1,용산,동인천,75
2,광운대,서동탄,32
3,서동탄,광운대,30
4,인천,양주,30
...,...,...,...
92,의정부,서울역,1
93,창동,소요산,1
94,창동,인천,1
95,천안,신창,1



[SAT] 총 87개 운행 구간
----------------------------------------------------------------------


,출발역,도착역,운행열차수
184,동인천,용산,72
185,용산,동인천,69
186,광운대,서동탄,34
187,서동탄,광운대,32
188,인천,양주,28
...,...,...,...
266,영등포,인천,1
267,용산,부평,1
268,인천,서울역,1
269,천안,구로,1



[END] 총 87개 운행 구간
----------------------------------------------------------------------


,출발역,도착역,운행열차수
97,동인천,용산,72
98,용산,동인천,69
99,광운대,서동탄,34
100,서동탄,광운대,32
101,인천,양주,28
...,...,...,...
179,영등포,인천,1
180,용산,부평,1
181,인천,서울역,1
182,천안,구로,1


##### 8. 운행횟수

In [7]:
# Context Manager 사용 (권장)
with sqlite3.connect(DB_PATH) as conn:
    query = f"""
            SELECT
                `역사명`,
                COUNT(DISTINCT `열차도착시간`) AS `도착횟수`,
                COUNT(DISTINCT `열차출발시간`) AS `출발횟수`
            FROM {TABLE}
            GROUP BY `역사명`
            ORDER BY `도착횟수` DESC
            """
    result = pd.read_sql(query, conn)
result.to_csv('query_result.csv', index=False, encoding='utf-8-sig')
result

,역사명,도착횟수,출발횟수
0,신도림,983,983
1,구로,981,979
2,영등포,970,971
3,노량진,956,956
4,대방,949,967
...,...,...,...
97,청산,74,72
98,전곡,74,72
99,신창,73,69
100,연천,37,37


##### 8. 배차시간

In [8]:
# Context Manager 사용 (권장)
with sqlite3.connect(DB_PATH) as conn:
    query = f"""
            SELECT
                `역사명`,
                `열차코드`,
                strftime("%H:%M:%S", `열차도착시간_dt`) AS `열차도착시간`,
                strftime("%H:%M:%S", `열차출발시간_dt`) AS `열차출발시간`,
                `출발역`,
                `도착역`,
                `방향`
            FROM {TABLE}
            WHERE `급행여부` = 0 AND `출발역` = "서울역"
            GROUP BY `역사명`, `열차코드`, `열차도착시간_dt`, `열차출발시간_dt`, `출발역`, `도착역`, `방향`
            ORDER BY `열차코드`, `출발역`, `도착역`, `열차도착시간`
            """
    result = pd.read_sql(query, conn)
result.to_csv('query_result.csv', index=False, encoding='utf-8-sig')
result.head(50)

,역사명,열차코드,열차도착시간,열차출발시간,출발역,도착역,방향
0,서울역,S401,None,05:20:00,서울역,서동탄,DOWN
1,남영,S401,05:22:30,05:23:00,서울역,서동탄,DOWN
2,용산,S401,05:25:00,05:25:30,서울역,서동탄,DOWN
3,노량진,S401,05:28:30,05:29:00,서울역,서동탄,DOWN
4,대방,S401,05:31:00,05:31:30,서울역,서동탄,DOWN
5,신길,S401,05:32:30,05:33:00,서울역,서동탄,DOWN
6,영등포,S401,05:35:00,05:35:30,서울역,서동탄,DOWN
7,신도림,S401,05:37:30,05:38:00,서울역,서동탄,DOWN
8,구로,S401,05:40:00,05:40:30,서울역,서동탄,DOWN
9,구로,S401,05:40:00,05:41:00,서울역,서동탄,DOWN
